In [11]:
import glob
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
from shapely.geometry import box
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")


def normalize(band):
    band_min, band_max = (band.min(), band.max())
    return ((band-band_min)/((band_max - band_min)))

def pct_clip(array, pct=[2,98]):
    array_min, array_max = np.nanpercentile(array, pct[0]), np.nanpercentile(array, pct[1])
    clip = (array - array_min) / (array_max - array_min)
    clip[clip>1] = 1
    clip[clip<0] = 0
    return clip

def chip_eo(img, envelope_gdf):
    with rasterio.open(img) as src:
        # Crop the image based on the envelope around the polygon
        window = src.window(*envelope_gdf.total_bounds)
        r, g, b = (pct_clip(normalize(src.read(k,window=window))) for k in (4, 3, 2))
    return np.dstack((r, g, b))

def chip_sar(img, envelope_gdf):
    with rasterio.open(img) as src:
        # Crop the image based on the envelope around the polygon
        window = src.window(*envelope_gdf.total_bounds)
        vh = pct_clip(normalize(src.read(1,window=window)))
        vv = pct_clip(normalize(src.read(2,window=window)))
        vhvv = vh / vv
    return np.dstack((vh, vv, vhvv))

In [13]:
eo_paths = [
    # "/filserver/user/imagery/Sentinel-2/2020/S2A_MSIL2A_20200626T104031_N0500_R008_T32VNM_20230507T041856_aligned_12b.tiff",
    "/filserver/user/imagery/Sentinel-2/2021/S2A_MSIL2A_20210624T105031_N0500_R051_T32VNM_20230318T092732_aligned_12b.tiff",
    # "/filserver/user/imagery/Sentinel-2/2022/S2B_MSIL2A_20220813T104629_N0400_R051_T32VNM_20220813T122218_aligned_12b.tiff",
    "/filserver/user/imagery/Sentinel-2/2023/S2B_MSIL2A_20230729T104629_N0509_R051_T32VNM_20230729T120309_aligned_12b.tiff"
               ]

sar_as_paths = [
    "/filserver/user/imagery/Sentinel-1/2021/S1A_IW_GRDH_1SDV_20210609T170237_20210609T170302_038266_0483FE_B5A6_CAL_COR_resampled_aligned.tiff",
    # "/filserver/user/imagery/Sentinel-1/2022/S1A_IW_GRDH_1SDV_20220710T170245_20220710T170310_044041_0541C2_45A7_CAL_COR_resampled_aligned.tiff",
    "/filserver/user/imagery/Sentinel-1/2023/S1A_IW_GRDH_1SDV_20230822T170252_20230822T170317_049991_0603A8_DE3F_CAL_COR_resampled_aligned.tiff"
               ]

sar_ds_paths = [
    "/filserver/user/imagery/Sentinel-1/2021/S1A_IW_GRDH_1SDV_20210604T053944_20210604T054009_038186_0481BA_DD5F_CAL_COR_resampled_aligned.tiff",
    # "/filserver/user/imagery/Sentinel-1/2022/S1A_IW_GRDH_1SDV_20220717T053953_20220717T054018_044136_0544A4_920C_CAL_COR_resampled_aligned.tiff",
    "/filserver/user/imagery/Sentinel-1/2023/S1A_IW_GRDH_1SDV_20230829T054001_20230829T054026_050086_0606E8_0239_CAL_COR_resampled_aligned.tiff"
               ]

In [25]:
year = "2021_2023"
# sensor = "s1"

change_geojson = f"./results/difference_{year}_qa_400.geojson"
gdf = gpd.read_file(change_geojson)
# gdf['abs_mean'] = gdf['mean'].abs()
# gdf['sum'] = gdf['abs_mean'] * gdf.geometry.area
# gdf = gdf.sort_values(by="sum", ascending=False).head(50).reset_index()
# gdf['real'] = ''
# gdf['what'] = ''
# gdf['method'] = sensor
display(gdf)

,DN,mean,abs_mean,real,method,what,geometry
0,-1,-0.663117,0.663117,0,s1s2_append,dammed water,"POLYGON ((588810.000 6673460.000, 588810.000 6..."
1,1,0.656016,0.656016,0,s1s2_append,dammed water (drained),"POLYGON ((580110.000 6655230.000, 580110.000 6..."
2,1,0.650475,0.650475,1,s1s2_append,construction,"POLYGON ((588590.000 6686150.000, 588590.000 6..."
3,1,0.649282,0.649282,0,s1s2_append,industrial activity,"POLYGON ((570460.000 6675120.000, 570460.000 6..."
4,1,0.629378,0.629378,1,s1s2_append,road,"POLYGON ((575160.000 6677350.000, 575160.000 6..."
...,...,...,...,...,...,...,...
395,-1,-0.600280,0.600280,None,s1s2_merge,None,"POLYGON ((587870.000 6673700.000, 587870.000 6..."
396,1,0.619217,0.619217,None,s1s2_merge,None,"POLYGON ((532890.000 6673180.000, 532890.000 6..."
397,-1,-0.570083,0.570083,None,s1s2_merge,None,"POLYGON ((574580.000 6676720.000, 574580.000 6..."
398,-1,-0.695929,0.695929,None,s1s2_merge,None,"POLYGON ((586240.000 6607060.000, 586240.000 6..."


In [41]:
buffer_m = 200
n_changes = 5
n_lookup_changes = 50
search_distance = buffer_m

eo_pairs = eo_paths[0:2]
sar_as_pairs = sar_as_paths[0:2]
sar_ds_pairs = sar_ds_paths[0:2]

for n in range(len(gdf)):
    print(n,"/",len(gdf))
    if gdf['real'].iloc[n] != "" and pd.notna(gdf['real'].iloc[n]):
        clear_output()
        next
    else:
        print(gdf.method.iloc[n])
        print(gdf.DN.iloc[n])
        polygon = gdf.geometry.iloc[n]
        x,y = polygon.exterior.xy
        print(f'{round(polygon.centroid.y)}, {round(polygon.centroid.x)}, 32')
        print(round(polygon.centroid.x), round(polygon.centroid.y))
        
        bounds = polygon.bounds
        buffer_box = list(map(sum, zip(bounds, (-buffer_m, -buffer_m, buffer_m, buffer_m))))
        bbox_geometry = gpd.GeoSeries([box(*buffer_box)])
        envelope_gdf = gpd.GeoDataFrame(geometry=bbox_geometry)

        fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(12,8))
        ax[0,0].set_ylabel(year.split("_")[0])
        ax[1,0].set_ylabel(year.split("_")[1])

        eo1 = chip_eo(eo_pairs[0], envelope_gdf).transpose(2,0,1)
        show(eo1, ax=ax[0,0], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))
        eo2 = chip_eo(eo_pairs[1], envelope_gdf).transpose(2,0,1)
        show(eo2, ax=ax[1,0], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))

        as1 = chip_sar(sar_as_pairs[0], envelope_gdf).transpose(2,0,1)
        show(as1, ax=ax[0,1], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))
        as2 = chip_sar(sar_as_pairs[1], envelope_gdf).transpose(2,0,1)
        show(as2, ax=ax[1,1], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))

        ds1 = chip_sar(sar_ds_pairs[0], envelope_gdf).transpose(2,0,1)
        show(ds1, ax=ax[0,2], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))
        ds2 = chip_sar(sar_ds_pairs[1], envelope_gdf).transpose(2,0,1)
        show(ds2, ax=ax[1,2], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))

        ax[0,0].plot(x,y, color='red')
        ax[1,0].plot(x,y, color='red')
        ax[0,1].plot(x,y, color='red')
        ax[1,1].plot(x,y, color='red')
        ax[0,2].plot(x,y, color='red')
        ax[1,2].plot(x,y, color='red')
             
        plt.show()
        real_change = input("True or False?")
        if real_change == 'next':
            next
        elif real_change == 'exit':
            break
        else:
            gdf['real'].iloc[n] = real_change
            what = input("What?")
            gdf['what'].iloc[n] = what
        gdf.to_file(f"./results/difference_{year}_qa_400.geojson", driver="GeoJSON")
        clear_output()

change_geojson = f"./results/difference_{year}_qa_400.geojson"
gdf = gpd.read_file(change_geojson)
gdf

,DN,mean,abs_mean,real,method,what,geometry
0,-1,-0.663117,0.663117,0,s1s2_append,dammed water,"POLYGON ((588810.000 6673460.000, 588810.000 6..."
1,1,0.656016,0.656016,0,s1s2_append,dammed water (drained),"POLYGON ((580110.000 6655230.000, 580110.000 6..."
2,1,0.650475,0.650475,1,s1s2_append,construction,"POLYGON ((588590.000 6686150.000, 588590.000 6..."
3,1,0.649282,0.649282,0,s1s2_append,industrial activity,"POLYGON ((570460.000 6675120.000, 570460.000 6..."
4,1,0.629378,0.629378,1,s1s2_append,road,"POLYGON ((575160.000 6677350.000, 575160.000 6..."
...,...,...,...,...,...,...,...
395,-1,-0.600280,0.600280,0,s1s2_merge,dammed water,"POLYGON ((587870.000 6673700.000, 587870.000 6..."
396,1,0.619217,0.619217,1,s1s2_merge,construction,"POLYGON ((532890.000 6673180.000, 532890.000 6..."
397,-1,-0.570083,0.570083,-1,s1s2_merge,deconstruction,"POLYGON ((574580.000 6676720.000, 574580.000 6..."
398,-1,-0.695929,0.695929,0,s1s2_merge,dammed water,"POLYGON ((586240.000 6607060.000, 586240.000 6..."


In [38]:
gdf[(gdf['real'] != "") & (gdf['real'].notnull())]

,DN,mean,abs_mean,real,method,what,geometry
0,-1,-0.663117,0.663117,0,s1s2_append,dammed water,"POLYGON ((588810.000 6673460.000, 588810.000 6..."
1,1,0.656016,0.656016,0,s1s2_append,dammed water (drained),"POLYGON ((580110.000 6655230.000, 580110.000 6..."
2,1,0.650475,0.650475,1,s1s2_append,construction,"POLYGON ((588590.000 6686150.000, 588590.000 6..."
3,1,0.649282,0.649282,0,s1s2_append,industrial activity,"POLYGON ((570460.000 6675120.000, 570460.000 6..."
4,1,0.629378,0.629378,1,s1s2_append,road,"POLYGON ((575160.000 6677350.000, 575160.000 6..."
...,...,...,...,...,...,...,...
295,-1,-0.668932,0.668932,0,s2,activity,"POLYGON ((520040.000 6699540.000, 520040.000 6..."
296,1,0.652631,0.652631,0,s2,vegetation,"POLYGON ((572640.000 6664500.000, 572640.000 6..."
297,-1,-0.683958,0.683958,0,s2,dammed water,"POLYGON ((587850.000 6673810.000, 587850.000 6..."
298,1,0.701482,0.701482,1,s2,road,"POLYGON ((587190.000 6672700.000, 587190.000 6..."


In [ ]:
     import os, geopandas as gpd

     event_tracks = gpd.GeoDataFrame(columns=['DN', 'mean',	'abs_mean', 'real',	'geometry', 'what'])

     for dir_root, dir_dir, js_file  in os.walk(directory):
       for f in js_file:
         if f.endswith(".json"):
         fname = os.path.basename(f)
         fname, fext = fname.split('.')
         print (fname)
         print ('Reading Track Path JSON ' + f)
         with open(f, 'r') as path_f:
           event_tracks = gpd.read_file(path_f).append(event_tracks)

In [48]:
columns=['DN', 'mean', 'abs_mean', 'real', 'geometry', 'method']
qa_gdf = gpd.GeoDataFrame(columns=columns)

qa_geojsons = glob.glob(f"./results/difference_*_{year}_norm_qa.geojson")
for qa_path in qa_geojsons:
    stem = Path(qa_path).stem
    if "s1s2" in stem:
        method = "_".join(stem.split("_")[1:3])
    else:
        method = stem.split("_")[1]
    print(method)
    tmp_gdf = gpd.read_file(qa_path)
    tmp_gdf["method"] = method
    qa_gdf = pd.concat([qa_gdf, tmp_gdf])
qa_gdf['what'] = ""
del qa_gdf['index']
qa_gdf.reset_index(drop=True, inplace=True)
qa_gdf

s1s2_append
s1s2_merge
s1
s2


,DN,mean,abs_mean,real,geometry,method,what
0,-1,-0.663117,0.663117,0,"POLYGON ((588810.000 6673460.000, 588810.000 6...",s1s2_append,
1,1,0.656016,0.656016,0,"POLYGON ((580110.000 6655230.000, 580110.000 6...",s1s2_append,
2,1,0.650475,0.650475,1,"POLYGON ((588590.000 6686150.000, 588590.000 6...",s1s2_append,
3,1,0.649282,0.649282,0,"POLYGON ((570460.000 6675120.000, 570460.000 6...",s1s2_append,
4,1,0.629378,0.629378,1,"POLYGON ((575160.000 6677350.000, 575160.000 6...",s1s2_append,
...,...,...,...,...,...,...,...
195,1,0.777725,0.777725,1,"POLYGON ((531640.000 6653050.000, 531640.000 6...",s2,
196,1,0.777599,0.777599,1,"POLYGON ((602040.000 6597970.000, 602040.000 6...",s2,
197,1,0.777319,0.777319,1,"POLYGON ((559460.000 6630140.000, 559460.000 6...",s2,
198,1,0.777313,0.777313,1,"POLYGON ((594240.000 6620100.000, 594240.000 6...",s2,


In [66]:
buffer_m = 300
n_changes = 5
n_lookup_changes = 50
search_distance = buffer_m

eo_pairs = eo_paths[0:2]

for n in range(len(qa_gdf)):
    print(n,"/",len(qa_gdf))
    if qa_gdf['what'].iloc[n] == "xxx":
        clear_output()
        next
    else:
        what = qa_gdf['what'].iloc[n]
        print(what)
        unique = list(qa_gdf['what'].unique())
        unique.sort()
        print(unique)
        print(qa_gdf.DN.iloc[n], qa_gdf.real.iloc[n])
        polygon = qa_gdf.geometry.iloc[n]
        x,y = polygon.exterior.xy
        print(f'{round(polygon.centroid.y)}, {round(polygon.centroid.x)}, 32')
        print(round(polygon.centroid.x), round(polygon.centroid.y))
        
        bounds = polygon.bounds
        buffer_box = list(map(sum, zip(bounds, (-buffer_m, -buffer_m, buffer_m, buffer_m))))
        bbox_geometry = gpd.GeoSeries([box(*buffer_box)])
        envelope_gdf = gpd.GeoDataFrame(geometry=bbox_geometry)

        fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(12,8))
        ax[0].set_ylabel(year.split("_")[0])
        ax[1].set_ylabel(year.split("_")[1])

        eo1 = chip_eo(eo_pairs[0], envelope_gdf).transpose(2,0,1)
        show(eo1, ax=ax[0], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))
        eo2 = chip_eo(eo_pairs[1], envelope_gdf).transpose(2,0,1)
        show(eo2, ax=ax[1], extent=(buffer_box[0],buffer_box[2],buffer_box[1],buffer_box[3]))

        ax[0].plot(x,y, color='red')
        ax[1].plot(x,y, color='red')
             
        plt.show()
        new_what = input("What?")
        if new_what == 'next':
            next
        elif new_what == 'exit':
            break
        elif new_what == '':
            qa_gdf['what'].iloc[n] = what
        else:
            qa_gdf['what'].iloc[n] = new_what
        qa_gdf.to_file(f"./results/difference_{year}_qa.geojson", driver="GeoJSON")
        clear_output()

geojson = f"./results/difference_{year}_qa.geojson"
qa_gdf = gpd.read_file(geojson)
qa_gdf

,DN,mean,abs_mean,real,method,what,geometry
0,-1,-0.663117,0.663117,0,s1s2_append,dammed water,"POLYGON ((588810.000 6673460.000, 588810.000 6..."
1,1,0.656016,0.656016,0,s1s2_append,dammed water (drained),"POLYGON ((580110.000 6655230.000, 580110.000 6..."
2,1,0.650475,0.650475,1,s1s2_append,construction,"POLYGON ((588590.000 6686150.000, 588590.000 6..."
3,1,0.649282,0.649282,0,s1s2_append,industrial activity,"POLYGON ((570460.000 6675120.000, 570460.000 6..."
4,1,0.629378,0.629378,1,s1s2_append,road,"POLYGON ((575160.000 6677350.000, 575160.000 6..."
...,...,...,...,...,...,...,...
195,1,0.777725,0.777725,1,s2,road,"POLYGON ((531640.000 6653050.000, 531640.000 6..."
196,1,0.777599,0.777599,1,s2,road,"POLYGON ((602040.000 6597970.000, 602040.000 6..."
197,1,0.777319,0.777319,1,s2,road,"POLYGON ((559460.000 6630140.000, 559460.000 6..."
198,1,0.777313,0.777313,1,s2,road,"POLYGON ((594240.000 6620100.000, 594240.000 6..."
